In [4]:
from huggingface_hub import HfApi

api = HfApi()
username="Min-kyu"
MODEL_NAME = "PJA_LLM_MODEL_8bit"

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('HUG_API_KEY')


# 자료 정리
# # api.create_repo(
# #     token=api_key,
# #     repo_id=f"{username}/{MODEL_NAME}",
# #     repo_type="model"
# # )

# api.upload_folder(
#     token=api_key,
#     repo_id=f"{username}/{MODEL_NAME}",
#     folder_path="./llama-3.1-korean-8b-hf-stable/checkpoint-45"
# )

# Test

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 학습된 모델
tokenizer = AutoTokenizer.from_pretrained(f"{username}/{MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(f"{username}/{MODEL_NAME}")

# model = torch.nn.DataParallel(model).cuda()

# input_text = """
# 이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.
# """

# inputs = tokenizer(input_text, return_tensors='pt')
# eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

# with torch.no_grad():
#     outputs = model.module.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=500, eos_token_id=eos_token_id)
#     print(tokenizer.decode(outputs[0]))


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/61.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [7]:
pip install bitsandbytes accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import gc

# 8bit 양자화 - 4bit보다 성능이 훨씬 좋음
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False
)

In [18]:
model = AutoModelForCausalLM.from_pretrained(
    f"{username}/{MODEL_NAME}",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
# 추론
input_text = """
이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.
"""

inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)
eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"].to('cuda'),
        max_new_tokens=256,
        eos_token_id=eos_token_id,
        do_sample=True,
        temperature=1.0,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)


이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.


Assistant: **프로젝트 상세 정보:**
{'project_summary': {'title': '3인칭 카드게임 기반 MMORPG', 'category': '게임', 'target_users': ['게임 애호가', '카드게임 유저', '모드 RPG 팬'],'main_purpose': '3인칭 카드게임 장르를 MMORPG로 변환하여 사용자가 즐길 수 있게 하기', 'key_features': [{'feature': '카드 드롭 시스템', 'description': '시간이 10초 지날 때마다 하나씩 카드가 드롭됨'}, {'feature': '스토리 모드', 'description': '선택하는 스토리라인에 따라 드롭되는 카드의 형식 달라짐'}, {'feature': '랜덤 가챠', 'description': '사용자의 운으로 인해 특정 카드 드림'}, {'feature': '시각적 이속도', 'description': '게임내내 스톤토포인트의 연속성으로 몰입도 향상'}], 'core_technologies': [{'category': 'Game', 'technologies': ['Unity', 'UEngine']}, {'
